# GST data acquisition

This notebook contains the code run on Rigetti hardware to carry out the pyGSTi gate set tomography.

The `smq1Q_XZ` model pack was chosen to reflect the fact that the Rigetti hardware natively supports $RX(\theta)$ and $RZ(\theta)$ for universal Clifford implementation.  Parallelization is needed because the compile time is the bottleneck on the execution. Activereset should also be used to decrease the execution time of the circuits.

With the goal of ultimately using PEC on a complete gateset, the `smq1Q-XYZI` modelpack was tested. However, compiling these circuits crashes the parallelpool several times, so this is still a remaining challenge.

In [1]:
#PyGSTi tools
import pygsti

#pre-built gateset to use
from pygsti.modelpacks import smq1Q_XYZI
from pygsti.data.dataset import DataSet
from pygsti.io.writers import write_dataset

#Rigetti tools
from pyquil import get_qc, Program
from pyquil.api import QCSClientConfiguration, local_forest_runtime
from pyquil.gates import RESET

configuration = QCSClientConfiguration.load()

#python helper libraries
from multiprocessing.pool import ThreadPool
import time
import numpy as np
import pickle

## Create experiment design
This is borrowed from the [tutorial notebook](https://github.com/pyGSTio/pyGSTi/blob/bfedc1de4d604f14b0f958615776fb80ddb59e33/jupyter_notebooks/Tutorials/algorithms/GST-Overview.ipynb) on GST with pyGSTi. The fiducials are a set of operators $\{F_i\}$ such that $F_i|\rho \rangle \rangle$ and $\langle \langle E | F_i$ form an 'informationally complete' set. The germs are a set of strings generated from the target gates with the desired lengths, specified by `maxLengths`. This set is suppoed to be 'amplificationally complete', making it as sensitive to every kind of error as possible. The tutorials are not very clear on how to make these, and they are hard-coded into the model packs.

In [2]:
target_model = smq1Q_XYZI.target_model()      # a Model object
prep_fiducials = smq1Q_XYZI.prep_fiducials()  # preparation circuits
meas_fiducials = smq1Q_XYZI.meas_fiducials()  # measurement circuits
germs = smq1Q_XYZI.germs()                    # circuits repeated to amplify noise
maxLengths = [1,2,4,8,16,32]
exp_design = pygsti.protocols.StandardGSTDesign(target_model, 
                                                prep_fiducials, 
                                                meas_fiducials,
                                                germs, maxLengths) #stores data structure of experiment

exp_design.all_circuits_needing_data
circuits = list(exp_design.all_circuits_needing_data) #Get list of circuits

Write empty protocol data, to be filled with experimental data

In [3]:
pygsti.io.write_empty_protocol_data('experiment_data/rigetti_XYZI_data', exp_design, clobber_ok=True)

This next cell loads the qpu. The `execution_timeout` and `compiler_timeout` were necessary to handle queueing in the parallel pool. 

In [ ]:
#Get QPU. Replace with simulator if no reservation
qpu = get_qc("Aspen-11", 
             client_configuration=configuration, 
             execution_timeout = 100000, 
             compiler_timeout = 100000) #I thought 27 hours seemed like a reasonable timeout

The `circ_fname` function is a terrible last-minute fix designed like a bijective hash between circuits and filenames within the system filename character limit. These files hold the pickled binaries.

In [ ]:
def circ_fname(circ):
    return "circuit_binaries/%s .circ"%str(circ).replace("-","").replace('\n','').replace('|','').replace(' ','').replace('G','')[8:]

shots = 1000 #number of shots for each circuit. This was the default value
num_circs = len(circuits)

start_time = time.time()

for (i,circ) in enumerate(circuits):
    #convert pyGSTi circuit to quil program. add active reset to speed up execution
    prog = Program(circ.convert_to_quil()).wrap_in_numshots_loop(shots)
    executable = qpu.compile(prog) #compile for target QPU
    
    with open(circ_fname(circ), "wb") as f:
        pickle.dump(executable, f) #dump binary for later use
    
    #A watched pot will still boil eventually
    print("finished ",i, " Done in ",(time.time()-start_time)/(i+1)*(num_circs-i), end='\r')
    
print(time.time() - start_time) #Around 1hr.

The compilation ended up being the bottleneck by far. The QPU runs jobs extremely quickly.

In [ ]:
results = []

#Run the program in the file identified by 'circ' and return results
def run(circ):
    
    with open(circ_fname(circ), "rb") as f: #hash circuit back to executable file
        executable = pickle.load(f)
        
    result = qpu.run(executable).readout_data.get("ro")
    zeros = len([i for i in result if i== [0]]) #count the number of zeros
    return zeros

#I still found issues with the parallel pool for execution,
#so I made this part iterative too
for (i,circ) in enumerate(circuits):
    print("Running", i,end='\r' ) #For sanity
    results.append(run(circ)) #add number of zeros from run to result

Store the results in a data set and write to disk

In [ ]:
#Dataset object acts like a 2-d dictionary, with keys as circuit names,
#followed by result outcome
data = DataSet()

for (circ, result) in zip(circuits, results):
    data.add_count_dict(circ, {'0':result, '1':shots-result})
    
#write dataset for later usage
write_dataset("experiment_data/rigetti_XYZI_data/data/dataset.txt", data)

On jupyterlabs the data folder needs to be compressed to download it

In [ ]:
%%bash

tar -cvf second_experiment_run_data.tar experiment_data circuit_binaries